- Sử dụng các phương pháp UAD như i-Forest, SR, STL, RC-Forest và Luminol để phát hiện bất thường trong dữ liệu giao dịch.
- Mỗi phương pháp sẽ tạo ra một tập nhãn ban đầu cho các điểm dữ liệu trong chuỗi thời gian. - *Labeling function (LFs)*

5 mô hình UAD: https://realseries.readthedocs.io/en/latest/pages/installation.html

(hiện tại mới fix được 4 model:  i-Forest, SR, RC-Forest và Luminol) - thêm 1 model LOF

# **1. Import**

In [1]:
!pip install luminol

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.5/824.5 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for future: filename=future-0.16.0-py3-none-any.whl size=487792 sha256=92c46122d8da15f415a689a2c5c0919c8b1b6869726b4470c0c53f666b2c6d89
  Stored in directory: /root/.cache/pip/wheels/99/10/cf/94af142fb7fe16e7327b17dc4d6d69ed041f8ec74b1fef10b2
Successfully built future
  Attempting uninstall: future
    Found existing installation: future 0.18.3
    Uninstalling future-0.18.3:
      Successfully uninstalled future-0.18.3


In [2]:
!pip install rrcf

  Preparing metadata (setup.py) ... done
  Created wheel for rrcf: filename=rrcf-0.4.4-py3-none-any.whl size=10601 sha256=970d7853b6d62805d891cd51c200b77bc4a044637345a1424d4715c81e5def57
  Stored in directory: /root/.cache/pip/wheels/51/d4/43/6ac8270752946ee2223022ce0ed0d66c9455d000e1facb53a6
Successfully built rrcf


In [3]:
!git clone https://github.com/xchuwenbo/realseries.git

Cloning into 'realseries'...
remote: Enumerating objects: 177, done.
remote: Total 177 (delta 0), reused 0 (delta 0), pack-reused 177
Receiving objects: 100% (177/177), 4.63 MiB | 9.06 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [4]:
#!apt-get install python2.7
#!ln -s /usr/bin/python2.7 /usr/bin/python
#!pip install statsmodels==0.10.2

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

from sklearn.decomposition import PCA
import realseries
from realseries.realseries.models.iforest import IForest
from realseries.realseries.models.sr import SpectralResidual
#from realseries.realseries.models.stl import STL
from realseries.realseries.models.lumino import Lumino
from realseries.realseries.models.rcforest import RCForest
from sklearn.svm import OneClassSVM

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
path = '/content/drive/MyDrive/Học kì 6/DS200/Dataset/'
path_split = '/content/drive/MyDrive/Học kì 6/DS200/Dataset/Split/'

# **2. Function**

In [37]:
def pro(data):
  data['Date'] = pd.to_datetime(data['Date'])
  data = data.sort_values('Date')
  data = data.set_index('Date')
  return data

def load_anomaly(path):
    train_data = pro(pd.read_csv(path + "train_data.csv"))
    train_labels = pd.read_csv(path + "train_labels.csv")
    test_data = pro(pd.read_csv(path + "test_data.csv"))
    test_labels = pd.read_csv(path + "test_labels.csv")
    return train_data, train_labels, test_data, test_labels


data = pd.read_csv(path+'Data_processed.csv')

In [38]:
from sklearn.neighbors import LocalOutlierFactor

In [39]:
def init_model():
    # Khởi tạo mô hình
    # IForest
    IF = IForest(n_estimators=1000, max_samples='auto', contamination=0.01, random_state=42)

    # SR (Spectral Residual)
    SR = SpectralResidual(
        series=None,  # Sẽ được cung cấp sau
        threshold=0.5,
        mag_window=200,
        score_window=10
    )

    # RC-Forest (Random Cut Forest)
    rcforest_model = RCForest(
        shingle_size=1,
        num_trees=100,
        tree_size=50
    )

    # Luminol
    lumino_model = Lumino()

    """# OneClassSVM
    OCsvm_model = OCSVM()"""

    clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1)

    return IF, SR, rcforest_model, lumino_model, clf

In [40]:
def train_model(IF, SR, rcforest_model, lumino_model, clf, input):
    # Huấn luyện mô hình
    # i-Forest
    IF.fit(input)
    if_score = IF.detect(input)
    thres = np.percentile(if_score, 99)
    # Xác định nhãn bất thường (0: bình thường, 1: bất thường)
    pred_label_if = (if_score > thres).astype(int)

    # SR (Spectral Residual)
    SR.series = np.array(input).flatten()
    pred_label_sr = SR.detect()
    sr_label = np.array(pred_label_sr["isAnomaly"])
    sr_score = np.array(pred_label_sr["score"])

    """# STL (Seasonal and Trend decomposition using Loess)
    decomp = STL_model.fit(
        input,
        period=90,
        lo_frac=0.6,
        lo_delta=0.01
    )
    resid = np.array(decomp["resid"].squeeze())
    stl_score = abs(resid)
    stl_score = (stl_score - np.min(stl_score)) / (np.max(stl_score) - np.min(stl_score))
    thres = stl_score.min() + 0.7 * (stl_score.max() - stl_score.min())
    stl_label = (stl_score > thres).astype(int)"""

    # RC-Forest (Random Cut Forest)
    rcforest_score = rcforest_model.detect(input)
    min_score = np.nanmin(rcforest_score)
    rcforest_score = np.nan_to_num(rcforest_score, nan=min_score)
    thres = np.percentile(rcforest_score, 99)
    pred_label_rcforest = (rcforest_score > thres).astype(int)

    # Luminol
    lumino_score = lumino_model.detect(np.array(input).flatten())
    thres = lumino_score.min() + 0.7 * (lumino_score.max() - lumino_score.min())
    pred_label_lumino = (lumino_score > thres).astype(int)

    """# OneClassSVM
    OCsvm_model.fit(input)
    svm_score = OCsvm_model.detect(input)
    thres = np.percentile(svm_score, 99)
    svm_label = (svm_score > thres).astype(int)"""

    # clf
    clf.fit(input)
    lof_scores = -clf.negative_outlier_factor_
    lof_scores = (lof_scores - np.min(lof_scores)) / (np.max(lof_scores) - np.min(lof_scores))
    pred_label_clf = (lof_scores > thres).astype(int)

    return pred_label_if, if_score, sr_label, sr_score, pred_label_rcforest, rcforest_score, pred_label_lumino, lumino_score, lof_scores, pred_label_clf

In [41]:
def add_results(df, pred_label_if, if_score, sr_label, sr_score, pred_label_rcforest, rcforest_score, pred_label_lumino, lumino_score, lof_scores, pred_label_clf):
    # Thêm cột nhãn bất thường và điểm số bất thường vào dữ liệu gốc
    df['IForest Label'] = pred_label_if
    df['IForest Score'] = if_score

    df['SR Label'] = sr_label
    df['SR Score'] = sr_score

    df['RC-Forest Label'] = pred_label_rcforest
    df['RC-Forest Score'] = rcforest_score

    df['Luminol Label'] = pred_label_lumino
    df['Luminol Score'] = lumino_score

    """df['STL Label'] = stl_label
    df['STL Score'] = stl_score

    df['OneClassSVM Label'] = svm_label
    df['OneClassSVM Score'] = svm_score"""

    df['LOF Label'] = pred_label_clf
    df['LOF Score'] = lof_scores

    return df

In [42]:
def uad(input, df):
    IF, SR, rcforest_model, lumino_model, clf  = init_model()
    #input = df.values.reshape(-1,1)
    pred_label_if, if_score, sr_label, sr_score, pred_label_rcforest, rcforest_score, pred_label_lumino, lumino_score, pred_label_clf, lof_scores = train_model(
        IF, SR, rcforest_model, lumino_model, clf, input)
    df_result = add_results(df, pred_label_if, if_score, sr_label, sr_score, pred_label_rcforest, rcforest_score,
                            pred_label_lumino, lumino_score, pred_label_clf, lof_scores)
    return df_result

# **3. Thực thi**

In [43]:
train_data, train_label, test_data, test_label = load_anomaly(path_split)

In [44]:
train_data.info()
train_data.head(2)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 14400 entries, 2023-06-12 04:00:00 to 2023-07-14 19:59:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    14400 non-null  float64
 1   High    14400 non-null  float64
 2   Low     14400 non-null  float64
 3   Close   14400 non-null  float64
 4   Volume  14400 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 675.0 KB


,Open,High,Low,Close,Volume
Date,,,,,
2023-06-12 04:00:00,180.04,181.73,180.04,181.73,3421
2023-06-12 04:03:00,181.69,181.69,181.60,181.62,89


## **3.1. Train set**

In [45]:
data = train_data.copy()

pca = PCA(n_components=1)
# Chuẩn hóa dữ liệu
normalized_data = (data - data.mean()) / data.std()
# Thực hiện PCA
transformed_data = pca.fit_transform(normalized_data)

data_uad = uad(transformed_data, data)

col_score = ['IForest Score', 'SR Score', 'RC-Forest Score', 'Luminol Score', 'LOF Score']
col_label = ['IForest Label', 'SR Label', 'RC-Forest Label', 'Luminol Label', 'LOF Label']

train_data_uad = data_uad.iloc[:, 5:].copy()

train_score_uad = data_uad[col_score].copy()
train_label_uad = data_uad[col_label].copy()

## **3.2 Test set**

In [46]:
df = test_data.copy()

pca = PCA(n_components=1)
# Chuẩn hóa dữ liệu
normalized_df = (df - df.mean()) / df.std()
# Thực hiện PCA
transformed_df = pca.fit_transform(normalized_df)

data_uad_test = uad(transformed_df, df)

col_score = ['IForest Score', 'SR Score', 'RC-Forest Score', 'Luminol Score', 'LOF Score']
col_label = ['IForest Label', 'SR Label', 'RC-Forest Label', 'Luminol Label', 'LOF Label']
test_data_uad = data_uad_test.iloc[:, 5:].copy()

test_score_uad = data_uad_test[col_score].copy()
test_label_uad = data_uad_test[col_label].copy()

# **4. Save data**

In [47]:
path_uad = '/content/drive/MyDrive/Học kì 6/DS200/Dataset/UAD/'

train_data_uad.to_csv(path_uad+'train_data_uad.csv')
train_score_uad.to_csv(path_uad+'train_score_uad.csv')
train_label_uad.to_csv(path_uad+'train_label_uad.csv')

test_data_uad.to_csv(path_uad+'test_data_uad.csv')
test_score_uad.to_csv(path_uad+'test_score_uad.csv')
test_label_uad.to_csv(path_uad+'test_label_uad.csv')

In [48]:
train_data_uad

,IForest Label,IForest Score,SR Label,SR Score,RC-Forest Label,RC-Forest Score,Luminol Label,Luminol Score,LOF Label,LOF Score
Date,,,,,,,,,,
2023-06-12 04:00:00,1,0.994409,0,0.000000,0,0.027778,0,0.0,0,0.477107
2023-06-12 04:03:00,0,0.777447,0,0.048261,0,0.083333,0,0.0,0,0.010792
2023-06-12 04:04:00,0,0.787222,0,0.059813,0,0.072917,0,0.0,0,0.047702
2023-06-12 04:05:00,0,0.773409,0,0.074640,0,0.182292,0,0.0,0,0.011290
2023-06-12 04:06:00,0,0.789549,0,0.081915,0,0.125000,0,0.0,0,0.044792
...,...,...,...,...,...,...,...,...,...,...
2023-07-14 19:53:00,0,0.054827,0,0.037256,0,0.020833,0,0.0,0,0.017757
2023-07-14 19:54:00,0,0.054202,0,0.033752,0,0.041667,0,0.0,0,0.008677
2023-07-14 19:55:00,0,0.055272,0,0.062805,0,0.071429,0,0.0,0,0.054263


In [49]:
train_score_uad

,IForest Score,SR Score,RC-Forest Score,Luminol Score,LOF Score
Date,,,,,
2023-06-12 04:00:00,0.994409,0.000000,0.027778,0.0,0.477107
2023-06-12 04:03:00,0.777447,0.048261,0.083333,0.0,0.010792
2023-06-12 04:04:00,0.787222,0.059813,0.072917,0.0,0.047702
2023-06-12 04:05:00,0.773409,0.074640,0.182292,0.0,0.011290
2023-06-12 04:06:00,0.789549,0.081915,0.125000,0.0,0.044792
...,...,...,...,...,...
2023-07-14 19:53:00,0.054827,0.037256,0.020833,0.0,0.017757
2023-07-14 19:54:00,0.054202,0.033752,0.041667,0.0,0.008677
2023-07-14 19:55:00,0.055272,0.062805,0.071429,0.0,0.054263


In [50]:
train_label_uad

,IForest Label,SR Label,RC-Forest Label,Luminol Label,LOF Label
Date,,,,,
2023-06-12 04:00:00,1,0,0,0,0
2023-06-12 04:03:00,0,0,0,0,0
2023-06-12 04:04:00,0,0,0,0,0
2023-06-12 04:05:00,0,0,0,0,0
2023-06-12 04:06:00,0,0,0,0,0
...,...,...,...,...,...
2023-07-14 19:53:00,0,0,0,0,0
2023-07-14 19:54:00,0,0,0,0,0
2023-07-14 19:55:00,0,0,0,0,0


In [51]:
test_data_uad

,IForest Label,IForest Score,SR Label,SR Score,RC-Forest Label,RC-Forest Score,Luminol Label,Luminol Score,LOF Label,LOF Score
Date,,,,,,,,,,
2023-06-12 04:01:00,0,0.711255,0,0.000000,0,0.080729,0,0.0,0,0.002295
2023-06-12 04:02:00,0,0.720084,0,0.029619,0,0.156250,0,0.0,0,0.001283
2023-06-12 04:09:00,0,0.747700,0,0.051127,0,0.093750,0,0.0,0,0.002089
2023-06-12 04:10:00,1,0.819496,0,0.063917,0,0.031250,0,0.0,0,0.001184
2023-06-12 04:11:00,1,0.822344,0,0.078114,0,0.305556,0,0.0,0,0.001272
...,...,...,...,...,...,...,...,...,...,...
2023-07-14 19:49:00,0,0.068025,0,0.051560,0,0.006944,0,0.0,0,0.000760
2023-07-14 19:51:00,0,0.067821,0,0.047230,0,0.000000,0,0.0,0,0.001052
2023-07-14 19:52:00,0,0.065187,0,0.048068,0,0.062500,0,0.0,0,0.002735


In [52]:
test_score_uad

,IForest Score,SR Score,RC-Forest Score,Luminol Score,LOF Score
Date,,,,,
2023-06-12 04:01:00,0.711255,0.000000,0.080729,0.0,0.002295
2023-06-12 04:02:00,0.720084,0.029619,0.156250,0.0,0.001283
2023-06-12 04:09:00,0.747700,0.051127,0.093750,0.0,0.002089
2023-06-12 04:10:00,0.819496,0.063917,0.031250,0.0,0.001184
2023-06-12 04:11:00,0.822344,0.078114,0.305556,0.0,0.001272
...,...,...,...,...,...
2023-07-14 19:49:00,0.068025,0.051560,0.006944,0.0,0.000760
2023-07-14 19:51:00,0.067821,0.047230,0.000000,0.0,0.001052
2023-07-14 19:52:00,0.065187,0.048068,0.062500,0.0,0.002735


In [53]:
test_label_uad

,IForest Label,SR Label,RC-Forest Label,Luminol Label,LOF Label
Date,,,,,
2023-06-12 04:01:00,0,0,0,0,0
2023-06-12 04:02:00,0,0,0,0,0
2023-06-12 04:09:00,0,0,0,0,0
2023-06-12 04:10:00,1,0,0,0,0
2023-06-12 04:11:00,1,0,0,0,0
...,...,...,...,...,...
2023-07-14 19:49:00,0,0,0,0,0
2023-07-14 19:51:00,0,0,0,0,0
2023-07-14 19:52:00,0,0,0,0,0
